#Install lib

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00


In [3]:
#!apt install git-lfs

#Enviroment

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible answers are allowed or not).
squad_v2 = True
#model_checkpoint = "xlm-roberta-base"
model_checkpoint = "deepset/xlm-roberta-base-squad2"
batch_size = 16

#Preprocessing the training data

In [7]:
import json
from pathlib import Path
import torch

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
def read_squad_train(path,data_index,context_size):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  size=0
  for group in squad_dict['data'][data_index-1:data_index]:
      for passage in group['paragraphs']:
        if size <= context_size:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                if answer['answer_start']>=0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)
          size+=1
        else: break

  return contexts, questions, answers
def read_squad_val(path):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)

  return contexts, questions, answers
def add_token_positions(encodings, answers):
  """
  Convert answer start- end from string to token
  """
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
    # if None, the answer passage has been truncated
    if start_positions[-1] is None:
        start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
        end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
def add_end_idx(answers, contexts):
  """
  Squad format chỉ có id start của câu trả lời
  -> Hàm này dùng để tạo id end của câu trả lời
  """
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two – fix this
    if context[start_idx:end_idx] == gold_text:
        answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

In [10]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_contexts, train_questions, train_answers = read_squad_train('/content/drive/MyDrive/CS221/DoAnCK/Dataset/squad_vi.json', 1,3500)
#val_contexts, val_questions, val_answers = read_squad_train('/content/drive/MyDrive/CS221/DoAnCK/Dataset/squad_vi.json', 3000, 3500)
add_end_idx(train_answers, train_contexts)
#add_end_idx(val_answers, val_contexts) 

#train
new_train_contexts=[]
new_train_questions=[]
new_train_answers=[]
for i in range(len(train_answers)):
  if train_answers[i]['answer_start']!=train_answers[i]['answer_end']:
    new_train_contexts.append(train_contexts[i])
    new_train_questions.append(train_questions[i])
    new_train_answers.append(train_answers[i])
(train_contexts, train_questions, train_answers)=(new_train_contexts, new_train_questions, new_train_answers)
(new_train_contexts, new_train_questions, new_train_answers)=(0,0,0)

#chia train val
val_contexts, val_questions, val_answers = train_contexts[12000:], train_questions[12000:], train_answers[12000:]
train_contexts, train_questions, train_answers = train_contexts[:12000], train_questions[:12000], train_answers[:12000]


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)


In [ ]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

#Load pretrain

In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
model

XLMRobertaForQuestionAnswering(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
 

#Train

In [12]:
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np

In [13]:
epochs = 10
batch_size=8
train_count = len(train_dataset)
val_count = len(val_dataset)
PATH = "/content/drive/MyDrive/CS221/DoAnCK/Model/best_model.model"

NameError: ignored

###Train with pytorch (Train thử với data mẫu gồm 1000 bộ câu hỏi)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(epochs):
  print("Epoch " + str(epoch) + ":")
  train_loss, val_loss = 0.0, 0.0
  train_correct, val_correct = 0, 0

  #train
  model.train()
  for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()
    train_loss += loss
    start, end = outputs.start_logits.argmax(dim=-1), outputs.end_logits.argmax(dim=-1)

    #print(f'start: {start_positions}, end: {end_positions} \nstart_pre: {start}, end_pre: {end}')
    for i in range(len(start)):
      if (start_positions[i] == start[i] and end_positions[i] == end[i]):
        train_correct += 1

  #Val
  #torch.cuda.empty_cache()
  # del outputs
  # model.eval()
  # for batch in val_loader:
  #   input_ids = batch['input_ids'].to(device)
  #   attention_mask = batch['attention_mask'].to(device)
  #   start_positions = batch['start_positions'].to(device)
  #   end_positions = batch['end_positions'].to(device)
  #   outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
  #   loss = outputs[0]
  #   val_loss += loss

  #   start, end = outputs.start_logits.argmax(dim=-1), outputs.end_logits.argmax(dim=-1)
  #   for i in range(len(start)):
  #     if (start_positions[i] == start[i] and end_positions[i] == end[i]):
  #       val_correct += 1

  print(f'\ttrain_loss: {train_loss/train_count}, train_EM: {round(train_correct/train_count, 3)}')#\n\tval_loss: {val_loss/val_count}, val_EM: {round(val_correct/val_count, 3)}')
  torch.save(model.state_dict(), PATH)

model.eval()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0:
	train_loss: 0.06970500946044922, train_EM: 0.782
Epoch 1:
	train_loss: 0.025197690352797508, train_EM: 0.91
Epoch 2:
	train_loss: 0.010780108161270618, train_EM: 0.951
Epoch 3:
	train_loss: 0.010792060755193233, train_EM: 0.961
Epoch 4:
	train_loss: 0.004178685136139393, train_EM: 0.988
Epoch 5:
	train_loss: 0.012848387472331524, train_EM: 0.956
Epoch 6:
	train_loss: 0.008505364879965782, train_EM: 0.961
Epoch 7:
	train_loss: 0.011643161065876484, train_EM: 0.956
Epoch 8:
	train_loss: 0.00583803141489625, train_EM: 0.975
Epoch 9:
	train_loss: 0.012150724418461323, train_EM: 0.962


XLMRobertaForQuestionAnswering(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
 

###Train with trainer (Chính thức)

In [ ]:
args = TrainingArguments(
    f"/content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [14]:
def f1_score(start_pred,end_pred,start_true,end_true):
  a=np.arange(int(start_pred),int(end_pred+1), step=1)
  b=np.arange(int(start_true),int(end_true+1), step=1)
  true_count = len(np.intersect1d(a,b))
  if ((int(end_true)-int(start_true)) == 0):
    recall = 0
  else:
    recall = true_count/(int(end_true)-int(start_true))
  if ((int(end_pred)-int(start_pred)) == 0):
    precision = 0
  else:
    precision= true_count/(int(end_pred)-int(start_pred))
  if (precision + recall == 0):
    return 0
  f1=2*(precision*recall)/(precision+recall)
  return f1

In [15]:
def compute_metrics(eval_pred):    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    count = 0
    for i in range(len(predictions[0])):
      if (predictions[0][i] == labels[0][i] and predictions[1][i] == labels[1][i]):
        count += 1
    f1 = 0
    for i in range(len(predictions[0])):
      f1 += f1_score(predictions[0][i], predictions[1][i], labels[0][i], labels[1][i])

    return {'exact_match': round(count/len(predictions[0]), 3), 'f1_score': round(f1/len(predictions[0]),3)} #metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/CS221/DoAnCK/Model/Roberta-ViSquad")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15000
  Number of trainable parameters = 277454594
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,1.479900,1.359919,52.696000,0.800000
2,1.099000,1.337566,54.340000,0.834000
3,0.805500,1.565913,54.751000,0.822000
4,0.606600,1.708217,54.545000,0.815000
5,0.461000,1.777834,54.751000,0.833000
6,0.361900,2.020115,54.340000,0.822000
7,0.284300,2.531387,55.932000,0.838000
8,0.220700,2.673394,55.829000,0.837000
9,0.163000,3.208999,55.419000,0.830000
10,0.149000,3.267859,55.470000,0.831000


Saving model checkpoint to /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-500
Configuration saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-1000
Configuration saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad/checkpoint-1000/token

###Evaluation

####Test

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-15000") 
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-15000")

In [ ]:
test_contexts, test_questions, test_answers = read_squad_val('/content/drive/MyDrive/CS221/DoAnCK/Dataset/Mailong25/train-v2.0.json')
add_end_idx(test_answers, test_contexts)
new_test_contexts=[]
new_test_questions=[]
new_test_answers=[]
for i in range(len(test_answers)):
  if test_answers[i]['answer_start']!=test_answers[i]['answer_end']:
    new_test_contexts.append(test_contexts[i])
    new_test_questions.append(test_questions[i])
    new_test_answers.append(test_answers[i])
(test_contexts, test_questions, test_answers)=(new_test_contexts, new_test_questions, new_test_answers)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True, return_offsets_mapping=True,)
add_token_positions(test_encodings, test_answers)
test_dataset = SquadDataset(test_encodings)

In [ ]:
args = TrainingArguments(
    f"/content/drive/MyDrive/CS221/DoAnCK/Logs/Roberta-ViSquad-test",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01
    #push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    #train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 985
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: offset_mapping. If offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.7535665035247803,
 'eval_exact_match': 0.241,
 'eval_f1_score': 0.527,
 'eval_runtime': 1037.9302,
 'eval_samples_per_second': 0.949,
 'eval_steps_per_second': 0.06}

####Predict

In [16]:
def Predict(contexts, questions, trainer):
  test_encodings = trainer.tokenizer(contexts, questions, truncation=True, padding=True, return_overflowing_tokens=True, return_offsets_mapping=True,)
  test_dataset = SquadDataset(test_encodings)
  
  result = trainer.predict(test_dataset, )
  predictions = np.argmax(result.predictions, axis =-1)
  metrics = result.metrics
  
  answers = []
  for i in range(len(contexts)):
    start = test_encodings['offset_mapping'][0][predictions[0][0]][0]
    end = test_encodings['offset_mapping'][0][predictions[1][0]][1]
    answers.append(contexts[i][start: end])
  return answers[0]

In [ ]:
a = Predict(['Nam sinh ra và lớn lên ở Thủ Đức. Nam học công nghệ thông tin'], ["Nam học gì"], trainer)
a

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.


' công nghệ thông tin'

In [ ]:
for i in range(0, 100, 10):
  print("Context:", test_contexts[i])
  print("Question:", test_questions[i])
  print("Trả lời:", Predict([test_contexts[i]], [test_questions[i]], trainer))
  print("\n==========")

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Context: Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .
Question: Ai là thủ tướng Ấn Độ hiện tại


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Trả lời: Narendra Damodardas Modi

Context: Chia sẻ với VnExpress sau khi được Forbes công nhận là tỷ phú USD với 1,3 tỷ USD tài sản , ông Trần Đình Long - Chủ tịch HĐQT Tập đoàn Hoà Phát , thú nhận đã lâu không làm việc để vì mục tiêu kiếm một con số cụ thể bao nhiêu tiền trong ngày .
Question: Ai là chủ tịch tập đoàn Hòa Phát


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.


...

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Trả lời: Ông Putin lần đầu tiên nắm quyền tổng thống Nga vào ngày 31/12/1999 , sau khi tổng thống đầu tiên của Nga , ông Boris Yeltsin

Context: Cùng với đó , Hội đồng quản trị FECON cũng đã bổ nhiệm ông Nguyễn Văn Thanh , Tổng giám đốc điều hành FECON chính thức đảm nhận vị trí Tổng giám đốc FECON từ ngày 8/11/2018 .
Question: tổng giám đốc công ty Fecon


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Trả lời: ,

Context: “ Vài năm nay , mọi người nhìn thép rất xấu , nhưng thép vẫn được coi là bánh mì của công nghiệp và tại những nước công nghiệp hoá mới như chúng ta , nhu cầu thép vẫn còn tăng ” , đó là chia sẻ chân tình của ông Trần Đình Long , Chủ tịch HĐQT Tập đoàn Hoà Phát khi nói về những mối quan tâm của dư luận thời gian gần đây trước những biến động lớn trong ngành thép lẫn tương lai của ngành này tại Việt Nam .
Question: chủ tịch tập đoàn Hòa Phát


Trả lời: ,



####Demo

In [ ]:
print("Context: My name is Sylvain.\nQuestion: What is your name?\nAnswer: Sylvain\nAnswer_start: 11")#\nAnswer_end: 18")

Context: My name is Sylvain.
Question: What is your name?
Answer: Sylvain
Answer_start: 11


In [ ]:
decode = tokenizer("My name is Sylvain.", "What is your name?", return_offsets_mapping=True)
for i in decode:
  print(f"{i}: {decode[i]}")

input_ids: [0, 2646, 9351, 83, 100973, 21845, 5, 2, 2, 4865, 83, 935, 9351, 32, 2]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
offset_mapping: [(0, 0), (0, 2), (2, 7), (7, 10), (10, 15), (15, 18), (18, 19), (0, 0), (0, 0), (0, 4), (4, 7), (7, 12), (12, 17), (17, 18), (0, 0)]


In [ ]:
add_token_positions(decode, [{'answer_start': 11, 'text': 'Sylvain', 'answer_end': 18}])
tokenizer.decode(decode["input_ids"])

'<s> My name is Sylvain.</s></s> What is your name?</s>'

In [ ]:
for i in decode:
  print(f"{i}: {decode[i]}")

input_ids: [0, 2646, 9351, 83, 100973, 21845, 5, 2, 2, 4865, 83, 935, 9351, 32, 2]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
offset_mapping: [(0, 0), (0, 2), (2, 7), (7, 10), (10, 15), (15, 18), (18, 19), (0, 0), (0, 0), (0, 4), (4, 7), (7, 12), (12, 17), (17, 18), (0, 0)]
start_positions: [4]
end_positions: [5]


In [ ]:
"My name is Sylvain."[11:18]

'Sylvain'

###Demo model

In [17]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer

In [ ]:
model1 = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/Roberta") 
tokenizer1 = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/Roberta")

#model2 = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/Minibert") 
#tokenizer2 = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/Minibert")

#model3 = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/distilBERT_0-3500") 
#tokenizer3 = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/distilBERT_0-3500")

#model4 = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/AlBERT") 
#tokenizer4 = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Model/AlBERT")

In [21]:
def Demo(path, args):
  model = AutoModelForQuestionAnswering.from_pretrained(path) 
  tokenizer = AutoTokenizer.from_pretrained(path)

  #Load data
  test_contexts, test_questions, test_answers = read_squad_val('/content/drive/MyDrive/CS221/DoAnCK/Dataset/Mailong25/train-v2.0.json')
  add_end_idx(test_answers, test_contexts)
  new_test_contexts=[]
  new_test_questions=[]
  new_test_answers=[]
  for i in range(len(test_answers)):
    if test_answers[i]['answer_start']!=test_answers[i]['answer_end']:
      new_test_contexts.append(test_contexts[i])
      new_test_questions.append(test_questions[i])
      new_test_answers.append(test_answers[i])
  (test_contexts, test_questions, test_answers)=(new_test_contexts, new_test_questions, new_test_answers)
  test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True, return_offsets_mapping=True,)
  add_token_positions(test_encodings, test_answers)
  test_dataset = SquadDataset(test_encodings)

  #Set trainer

  trainer = Trainer(
    model,
    args,
    #train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

  #Predict
  context = []
  question = []
  answer = []
  for i in range(0, 100, 10):
    context.append(test_contexts[i])
    question.append(test_questions[i])
    answer.append(Predict([test_contexts[i]], [test_questions[i]], trainer))
    
  return context, question, answer

In [19]:
args = TrainingArguments(
    f"/content/drive/MyDrive/CS221/DoAnCK/Logs/test",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01
    #push_to_hub=True,
)

Model1

In [27]:
a, b, c = Demo("/content/drive/MyDrive/CS221/DoAnCK/Model/Roberta", args)
for i in range(len(a)):
  print(f"Ex {i}: \n\tContext: {a[i]}\n\tQuestion: {b[i]}\n\tAnswer: {c[i]}")

Ex 0: 
	Context: Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .
	Question: Ai là thủ tướng Ấn Độ hiện tại
	Answer: Narendra Damodardas Modi
Ex 1: 
	Context: Chia sẻ với VnExpress sau khi được Forbes công nhận là tỷ phú USD với 1,3 tỷ USD tài sản , ông Trần Đình Long - Chủ tịch HĐQT Tập đoàn Hoà Phát , thú nhận đã lâu không làm việc để vì mục tiêu kiếm một con số cụ thể bao nhiêu tiền trong ngày .
	Question: Ai là chủ tịch tập đoàn Hòa Phát
	Answer:  ông Trần Đình Long
Ex 2: 
	Context: Nhà Trắng ngày 29/4 cho biết , Tổng thống Mỹ Donald Trump đã mời Tổng thống Philippines Rodrigo Duterte tới thăm Mỹ .
	Question: Ai là tổng thống Philippines
	Answ

Model 2

In [28]:
a, b, c = Demo("/content/drive/MyDrive/CS221/DoAnCK/Model/Minibert", args)
for i in range(len(a)):
  print(f"Ex {i}: \n\tContext: {a[i]}\n\tQuestion: {b[i]}\n\tAnswer: {c[i]}")

loading configuration file /content/drive/MyDrive/CS221/DoAnCK/Model/Minibert/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/CS221/DoAnCK/Model/Minibert",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "name": "Roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/MyDrive/CS221/DoAnCK/Model/Minibert/pytorch_mod

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


...

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Ex 0: 
	Context: Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .
	Question: Ai là thủ tướng Ấn Độ hiện tại
	Answer: Narendra Damodardas Modi
Ex 1: 
	Context: Chia sẻ với VnExpress sau khi được Forbes công nhận là tỷ phú USD với 1,3 tỷ USD tài sản , ông Trần Đình Long - Chủ tịch HĐQT Tập đoàn Hoà Phát , thú nhận đã lâu không làm việc để vì mục tiêu kiếm một con số cụ thể bao nhiêu tiền trong ngày .
	Question: Ai là chủ tịch tập đoàn Hòa Phát
	Answer: Chia sẻ với VnExpress
Ex 2: 
	Context: Nhà Trắng ngày 29/4 cho biết , Tổng thống Mỹ Donald Trump đã mời Tổng thống Philippines Rodrigo Duterte tới thăm Mỹ .
	Question: Ai là tổng thống Philippines
	An

Model 3

In [29]:
a, b, c = Demo("/content/drive/MyDrive/CS221/DoAnCK/Model/distilBERT_0-3500", args)
for i in range(len(a)):
  print(f"Ex {i}: \n\tContext: {a[i]}\n\tQuestion: {b[i]}\n\tAnswer: {c[i]}")

loading configuration file /content/drive/MyDrive/CS221/DoAnCK/Model/distilBERT_0-3500/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/CS221/DoAnCK/Model/distilBERT_0-3500",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "eos_token_ids": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "vocab_size": 30522
}

loading weights file /content/drive/MyDrive/CS221/DoAnCK/Model/distilBERT_0-3500/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQues

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


...

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.


Ex 0: 
	Context: Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .
	Question: Ai là thủ tướng Ấn Độ hiện tại
	Answer: Narendra Damodardas Modi
Ex 1: 
	Context: Chia sẻ với VnExpress sau khi được Forbes công nhận là tỷ phú USD với 1,3 tỷ USD tài sản , ông Trần Đình Long - Chủ tịch HĐQT Tập đoàn Hoà Phát , thú nhận đã lâu không làm việc để vì mục tiêu kiếm một con số cụ thể bao nhiêu tiền trong ngày .
	Question: Ai là chủ tịch tập đoàn Hòa Phát
	Answer: Forbes
Ex 2: 
	Context: Nhà Trắng ngày 29/4 cho biết , Tổng thống Mỹ Donald Trump đã mời Tổng thống Philippines Rodrigo Duterte tới thăm Mỹ .
	Question: Ai là tổng thống Philippines
	Answer: Donald Tr

Model 4

In [30]:
a, b, c = Demo("/content/drive/MyDrive/CS221/DoAnCK/Model/AlBERT", args)
for i in range(len(a)):
  print(f"Ex {i}: \n\tContext: {a[i]}\n\tQuestion: {b[i]}\n\tAnswer: {c[i]}")

loading configuration file /content/drive/MyDrive/CS221/DoAnCK/Model/AlBERT/config.json
Model config AlbertConfig {
  "_name_or_path": "/content/drive/MyDrive/CS221/DoAnCK/Model/AlBERT",
  "architectures": [
    "AlbertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "eos_token_ids": null,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_siz

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


...

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `AlbertForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `AlbertForQuestionAnswering.forward`,  you can safely ignore this message.


Ex 0: 
	Context: Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .
	Question: Ai là thủ tướng Ấn Độ hiện tại
	Answer: Narendra Damodardas
Ex 1: 
	Context: Chia sẻ với VnExpress sau khi được Forbes công nhận là tỷ phú USD với 1,3 tỷ USD tài sản , ông Trần Đình Long - Chủ tịch HĐQT Tập đoàn Hoà Phát , thú nhận đã lâu không làm việc để vì mục tiêu kiếm một con số cụ thể bao nhiêu tiền trong ngày .
	Question: Ai là chủ tịch tập đoàn Hòa Phát
	Answer:  Forbes
Ex 2: 
	Context: Nhà Trắng ngày 29/4 cho biết , Tổng thống Mỹ Donald Trump đã mời Tổng thống Philippines Rodrigo Duterte tới thăm Mỹ .
	Question: Ai là tổng thống Philippines
	Answer:  Rodrigo Dute

In [44]:
for i in range(10):
  print()